In [1]:
from bglog import BGLog, get_embedding_layer
import numpy as np
import tensorflow as tf
tf.random.set_seed(123)

In [2]:
bglog = BGLog(save_padded_num_sequences=False, load_from_pkl=True)

In [3]:
train_test = bglog.get_tensor_train_test(ablation=1000)
train_data, test_data = train_test

padded_num_seq_df loaded from data\bgl_padded_num_seq_df.pkl
trained tokenizer, tk, loaded from data\bgltk.pkl
train_0:, 800
test_0:, 200
train_1:, 800
test_1:, 200
train_2:, 800
test_2:, 200
train_3:, 800
test_3:, 102
4 class does not have 800 records, it has only 628 records
test_4:, 0
5 class does not have 800 records, it has only 165 records
5 class does not have 200 records, it has only 165 records
6 class does not have 800 records, it has only 75 records
6 class does not have 200 records, it has only 75 records
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>


In [4]:
def model(conv1d_set1 = 3, conv1d_set2 = 3, dense_neurons=2048, filters=64,
            kernel_size=3,maxpool_1=True,epochs=25, dense_activation='relu'):
    embedding_weights, vocab_size, char_onehot = get_embedding_layer(bglog)
    B = train_data.element_spec[0].shape[0]
#     inputs = tf.keras.layers.Input(batch_shape=(B, train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
    inputs = tf.keras.layers.Input(shape=(train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
    x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                    output_dim=vocab_size,
                                    input_length=train_data.element_spec[0].shape[2],
                                    weights = [embedding_weights],
                                    )(inputs)
    for _ in range(conv1d_set1):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    if maxpool_1:
        x = tf.keras.layers.MaxPooling2D(pool_size=(1, train_data.element_spec[0].shape[2]))(x)
        x = tf.reshape(x, (B, train_data.element_spec[0].shape[1], filters))        
        for _ in range(conv1d_set2):
            x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        x = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )(x)
        x = tf.reshape(x, (B, filters))
    if not maxpool_1:
        x = tf.keras.layers.Flatten()(x)
    if dense_activation is None:
        x = tf.keras.layers.Dense(dense_neurons)(x)
    else:
        x = tf.keras.layers.Dense(dense_neurons, activation=dense_activation)(x)
    outputs = tf.keras.layers.Dense(train_data.element_spec[1].shape[1], activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 
    return model, hist

In [5]:
# we  feed  xi  to  a dense layer h to get the log-sequence representation zi∈RD:
#     zi= h(xi) =σ(Whxi+bh) ............................(2)
# in our case zi can be obtained from the dense layer before the softmax
# Lets see how to ger it from the train mode

In [6]:
# we pre-train the model with labeled known intent samples. 
# In order to better reflect the effectiveness of the learned decision boundary, 
# we learn the feature representation zi with the simple softmax loss Ls to perform classification:

# trained_model, hist = model(epochs=6,)

In [7]:
# trained_model, hist = model(epochs=6, dense_neurons=64)

In [8]:
# Learn the decision boundary of each class constraining the known labels within a ball area
# how to get zi and how to know that zi belongs to which yi ?
# from there we will have to calculate the Ck , centroid for the class k

In [9]:
# trained_model.layers

In [10]:
# dense_6 = trained_model.get_layer(index=(len(trained_model.layers)-1))
# print(dense_6)

In [11]:
#This is the log sequence embedding from the last layer
# we can treat this as the features from the logs
# dense_6.output

In [12]:
# Then, we use the pre-trained model to extract intent features for 
# learning the decision boundary

In [13]:
class LogLineEncoder(tf.keras.Model):
    def __init__(self, num_of_conv1d=3,  
                 filters=64,
                 kernel_size=3, ):
        super().__init__()            
        self.num_of_conv1d = num_of_conv1d       
        self.filters = filters
        self.kernel_size = kernel_size           
        self.embedding_weights, self.vocab_size, self.char_onehot = get_embedding_layer(bglog)       
        
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size+1,
                                    output_dim=self.vocab_size,
                                    input_length=train_data.element_spec[0].shape[2],
                                    weights = [self.embedding_weights],
                                    )
        self.conv1d_layers = [tf.keras.layers.Conv1D(filters=filters, 
                                                kernel_size=kernel_size, 
                                                padding='same')  
                       for _ in range(self.num_of_conv1d)]
        self.maxpool2d = tf.keras.layers.MaxPooling2D(
            pool_size=(1, train_data.element_spec[0].shape[2]))
                  
        
    def call(self, inputs):
        x = self.embedding(inputs)
        for conv1d_layer in self.conv1d_layers:
            x = conv1d_layer(x)
        x = self.maxpool2d(x)
        x = tf.reshape(x, (inputs.shape[0], inputs.shape[1], self.filters))
        return x
    
    

# 
line_encoder =   LogLineEncoder()
# the model doesn't have a state unless it is called at least once
# in order to initialize the model we need a sample data 
sample_train_data = next(iter(train_data))
sample_x_train = sample_train_data[0]
print('sample_x_train.shape:', sample_x_train.shape)
# now we will initialize the model with the sample data
loglineEmbedding = line_encoder(sample_x_train)
print('loglineEmbedding.shape:', loglineEmbedding.shape)
# Now the model have a state and can be inspected        
line_encoder.summary()

vocab_size: 50
sample_x_train.shape: (32, 32, 64)
loglineEmbedding.shape: (32, 32, 64)
Model: "log_line_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2550      
_________________________________________________________________
conv1d (Conv1D)              multiple                  9664      
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  12352     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  12352     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
Total params: 36,918
Trainable params: 36,918
Non-trainable params: 0
_________________________________________________________________


LOG SEQUENCE EMBEDDING TAKES LOGLINE EMBEDDING AS INPUT

In [14]:
class LogSeqEncoder(tf.keras.Model):
    
    def __init__(self, num_of_conv1d=3,  filters=64,
                 kernel_size=3, maxpool_1=True,
                 dense_neurons=16, dense_activation='relu',):
        super().__init__()
        self.num_of_conv1d = num_of_conv1d
        self.dense_neurons = dense_neurons
        self.filters = filters
        self.kernel_size = kernel_size
        self.maxpool_1 = maxpool_1
        self.dense_activation = dense_activation
        self.conv1d_layers = [tf.keras.layers.Conv1D(filters=filters, 
                                                kernel_size=kernel_size, 
                                                padding='same')  
                       for _ in range(self.num_of_conv1d)]
        self.maxpool1d = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )
        
        self.Dense = tf.keras.layers.Dense(self.dense_neurons, 
                                           activation=self.dense_activation)
       
        
    def call(self, inputs):       
        for conv1d_layer in self.conv1d_layers:
            x = conv1d_layer(inputs)
        x = self.maxpool1d(x)        
        x = tf.reshape(x, (inputs.shape[0], self.filters))
        x = self.Dense(x)
        return x
    
    

logSeqencer =   LogSeqEncoder()
# the model doesn't have a state unless it is called at least once
logSeqEmbedding = logSeqencer(loglineEmbedding)
print('logSeqEmbedding.shape:', logSeqEmbedding.shape)
# Now the model have a state and can be inspected        
logSeqencer.summary()

logSeqEmbedding.shape: (32, 16)
Model: "log_seq_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            multiple                  12352     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  12352     
_________________________________________________________________
conv1d_5 (Conv1D)            multiple                  12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1040      
Total params: 38,096
Trainable params: 38,096
Non-trainable params: 0
_________________________________________________________________


In [15]:
# sample_x_train

In [16]:
class LogClassifier(tf.keras.Model):
    
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        self.log_line_encoder = LogLineEncoder()
        self.log_seq_encoder = LogSeqEncoder()
        self.classifier = tf.keras.layers.Dense(
            train_data.element_spec[1].shape[1], activation='softmax')
#         self.extract_feature = extract_feature
    
    def call(self, inputs, extract_feature=False,):
#         x_data, y_data = inputs
        x = self.log_line_encoder(inputs)
        seq_embedding = self.log_seq_encoder(x)
        
        if  extract_feature:
            output = seq_embedding
        else:
            output = self.classifier(seq_embedding)
        return output
    
log_classifier = LogClassifier()
log_classifier(sample_x_train)        

vocab_size: 50


<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[0.3362508 , 0.18392177, 0.25280538, 0.22702198],
       [0.3259221 , 0.17259048, 0.25523707, 0.24625033],
       [0.33589986, 0.17154121, 0.2527363 , 0.23982258],
       [0.31977898, 0.18768497, 0.24569227, 0.2468438 ],
       [0.32673177, 0.19432285, 0.24594125, 0.23300415],
       [0.33127734, 0.17444712, 0.24941857, 0.24485695],
       [0.34088168, 0.18399805, 0.24740067, 0.22771955],
       [0.35052803, 0.17589073, 0.25080392, 0.2227773 ],
       [0.3301531 , 0.19087662, 0.25464252, 0.22432776],
       [0.32286835, 0.19903256, 0.23758425, 0.24051486],
       [0.33854622, 0.17458344, 0.24335903, 0.24351132],
       [0.34005564, 0.17529859, 0.24966821, 0.23497763],
       [0.3234403 , 0.20333713, 0.24460691, 0.22861567],
       [0.3295585 , 0.17948176, 0.24202996, 0.24892984],
       [0.33527887, 0.18222594, 0.23787557, 0.2446196 ],
       [0.33527887, 0.18222594, 0.23787557, 0.2446196 ],
       [0.33407888, 0.19174811, 0.25195

In [17]:
# the classifier assigned low probability to all the classes since it is untrained
# TODO: the mode should accept a single sequence. At present it is accepting only a batch

In [18]:
log_classifier.summary()

Model: "log_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
log_line_encoder_1 (LogLineE multiple                  36918     
_________________________________________________________________
log_seq_encoder_1 (LogSeqEnc multiple                  38096     
_________________________________________________________________
dense_2 (Dense)              multiple                  68        
Total params: 75,082
Trainable params: 75,082
Non-trainable params: 0
_________________________________________________________________


In [19]:
# This is to check that the model's built in  complile and fit is working well
log_classifier.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = log_classifier.fit(train_data, validation_data=test_data, epochs=1) 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [20]:
# now after the training the predeicitoin will show higher probability to the 
# a class and lesser probability to other classes
log_classifier(sample_x_train)      

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[7.60580369e-05, 2.69009047e-06, 1.02337904e-03, 9.98897791e-01],
       [9.99056160e-01, 5.29905141e-04, 8.74223915e-05, 3.26555717e-04],
       [7.98910558e-01, 3.62408906e-02, 2.36578919e-02, 1.41190618e-01],
       [9.99064624e-01, 5.99953171e-04, 8.24591043e-05, 2.53109116e-04],
       [2.71843077e-04, 1.46409031e-03, 9.95238781e-01, 3.02519766e-03],
       [1.31694088e-03, 9.98520672e-01, 3.33402349e-05, 1.29078122e-04],
       [2.46036245e-04, 7.63872988e-04, 9.91047323e-01, 7.94285163e-03],
       [4.08728738e-05, 1.02642689e-05, 8.79703730e-04, 9.99069154e-01],
       [6.60562073e-05, 4.30192704e-05, 3.03573534e-03, 9.96855140e-01],
       [2.94254348e-03, 4.72355224e-02, 8.44164714e-02, 8.65405440e-01],
       [1.23585924e-03, 9.98609424e-01, 3.16517253e-05, 1.22916113e-04],
       [9.98518765e-01, 8.18154440e-05, 4.18232870e-04, 9.81201883e-04],
       [4.19493852e-04, 2.11013737e-03, 9.60133038e-03, 9.87868965e-01],
  

In [21]:
features = log_classifier(sample_x_train, extract_feature=True ) 
print('features.shape:', features.shape)
features[:3]

features.shape: (32, 16)


<tf.Tensor: shape=(3, 16), dtype=float32, numpy=
array([[ 0.        ,  0.5871825 ,  0.        ,  0.        ,  0.34673932,
         0.        ,  8.606613  ,  2.5149047 ,  0.        ,  0.        ,
        10.062949  ,  3.454819  , 12.330506  ,  0.        ,  0.        ,
         0.        ],
       [ 5.323514  ,  8.875267  ,  0.        ,  0.        ,  4.44558   ,
         0.        ,  5.3081484 , 10.493349  ,  0.        ,  7.499896  ,
         1.7070572 ,  3.0301955 ,  3.6977694 ,  0.        ,  0.        ,
         0.        ],
       [ 3.8707469 ,  5.8908477 ,  0.        ,  0.        ,  2.8829777 ,
         0.        ,  5.878939  ,  7.3715367 ,  0.        ,  5.5164213 ,
         3.551658  ,  4.6453    ,  7.011702  ,  0.        ,  0.        ,
         0.        ]], dtype=float32)>

In [22]:
np.zeros((2, 2))

array([[0., 0.],
       [0., 0.]])

In [23]:
for batch in train_data:
    x_train, y_train = batch
    print(x_train)
    print(y_train)
    break

tf.Tensor(
[[[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 ...

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[ 3 20  3 ...  3  8  9]
  [10  8  7 ...  8  4 14]
  [12  3  6 ... 12  6 18]
  ...
  [ 3 20  3 ...  3  8  9]
  [10  8  7 ...  8  4 14]
  [12  3  6 ... 12  6 18]]]

In [24]:
centroids = np.zeros((train_data.element_spec[1].shape[1],   16))
print('centriods initialized:', centroids)
total_labels = np.zeros(train_data.element_spec[1].shape[1]) # it was 4
# total_labels[2] += 1
# total_labels[2] += 1
print('total_labels initialized:', total_labels)
for batch in train_data: # Remember <BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
    logseq_batch, label_batch = batch
    # (32, 32, 64), (32, 4)
    features = log_classifier(logseq_batch, extract_feature=True )
    # (32, 16) features - 32 sequence of line each haaving 64 characrers
    # produces a feaure vector of dimension 16. 
    for i in range(len(label_batch)): # (32, 4) --> here length is 32
        label = label_batch[i] # label looks like [0 0 0 1]
        numeric_label = np.argmax(label) # index position of the label = 3 , so it is actually class =3
        ##total_labels = [0 0 0 0] each col representing a class 
        ## count the number for each class
        total_labels[numeric_label] += 1 
        centroids[numeric_label] += features[i] 
        # each row index in the centroid array is a class
        # we add first identify the feature belonging to which class by the numeric_label
        # Then add all the features belonging to the class in the corresponding row of the centroid array

### shape of centroids is (4, 16) whereas shape of total_labels is (1, 4)
### reshape the total_labels as 4,1 ==> [[0], [0], [0], [0]]==> 4 rows 
## so that we can divide the centroids array by the total_labels
total_label_reshaped = np.reshape(total_labels, (train_data.element_spec[1].shape[1], 1))
centroids /= total_label_reshaped
print('centroids:',centroids)
print('total_labels:',total_label_reshaped)

centriods initialized: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
total_labels initialized: [0. 0. 0. 0.]
centroids: [[ 4.46113159  8.66170288  0.          0.          4.52110077  0.
   5.7082959   9.75568176  0.          7.23651123  2.17397827  3.18548706
   4.32208893  0.          0.          0.        ]
 [ 7.16514893  2.00884033  0.          0.          1.42561722  0.
   2.87550415  8.80388062  0.          6.78546265  0.40732346  6.28099854
   7.50482544  0.          0.          0.03961719]
 [ 0.19467258  0.02143901  0.          0.          1.57748123  0.
   4.63262024  0.77323769  0.          3.89771362  6.93959778  7.18499756
   7.57732971  0.          0.          4.32237976]
 [ 0.56491795  0.38422638  0.          0.          0.34734833  0.
   7.69043091  3.06733826  0.          0.70252678  8.42969543  4.10188232
  11.404

In [25]:
# to understand the np divide operation
n1 = np.arange(12)
n2 = n1.reshape((3, 4))
print('n2 is 3 rows and each row has a vector of 4 cols:\n', n2)
n3 = np.array([2, 2, 2])
print('n3, is one single row:\n ', n3)
n4 = np.reshape(n3, (3, 1))
print('one single row is now converted to three rows, n4:\n', n4)
print(f'now division between n2: {n2.shape} and n4: {n4.shape} is possible')
n2/n4

n2 is 3 rows and each row has a vector of 4 cols:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
n3, is one single row:
  [2 2 2]
one single row is now converted to three rows, n4:
 [[2]
 [2]
 [2]]
now division between n2: (3, 4) and n4: (3, 1) is possible


array([[0. , 0.5, 1. , 1.5],
       [2. , 2.5, 3. , 3.5],
       [4. , 4.5, 5. , 5.5]])

In [26]:
#take zi and a ck 
# from sample_x_train the first sample belongs to class 3
print('sample_x_train[0]', sample_x_train[0])
sample_y_train = sample_train_data[1]
print('sample_y_train[0]', sample_y_train[0])
print('feature for the same:', features[0])
print('centroid for the class 3 :', centroids[3])

sample_x_train[0] tf.Tensor(
[[10  5 14 ...  2  2 11]
 [10  5 14 ... 26  2 19]
 [10  5 14 ... 23  2 23]
 ...
 [ 4 27 19 ...  9  0  0]
 [12  3  6 ... 12  6 18]
 [ 4 27 19 ...  9  0  0]], shape=(32, 64), dtype=int32)
sample_y_train[0] tf.Tensor([0. 0. 0. 1.], shape=(4,), dtype=float32)
feature for the same: tf.Tensor(
[3.7446022 0.5150217 0.        0.        0.6712336 0.        2.7540977
 5.176922  0.        4.8775744 2.0351896 6.470668  7.589819  0.
 0.        0.4365957], shape=(16,), dtype=float32)
centroid for the class 3 : [ 0.56491795  0.38422638  0.          0.          0.34734833  0.
  7.69043091  3.06733826  0.          0.70252678  8.42969543  4.10188232
 11.40448486  0.          0.          0.37472347]


In [27]:
# eucladian distance 
#first sample belonging to class 3 = z_0_3, dimension of it is same as the dense neuron=16
z_0_3= features[0] # [16], earlier [2048]
C_3 = centroids[3] # [16], earlier [2048]
ED = np.sum(np.square(z_0_3 - C_3 ))
print('eucladian distance:', ED)
# InvalidArgumentError: Incompatible shapes: [32,64] vs. [2048] [Op:Sub]

eucladian distance: 117.537506


In [28]:
# lets f1 is one feature from a batch of 32
# instead of 16, lets say the dimension of f1 is 5
f1 = np.arange(20, 25)
# a1 = np.reshape(a1, (4, 3) )
print('one feature with dimension 5 , f1:\n', f1)
### lets say ctd is the centroids with 4 class , each row is a class 
ctd = np.arange(20)
ctd = np.reshape(ctd, (4, 5))
print('centroid, ctd with 4 class :\n', ctd)
sub_result = f1 - ctd
print()
print('sub_result, for each feature we have 4 rows after substraction:\n', sub_result)
print('the 4 rows of the result are the distance of the feature from the centroid of each class')
for i, row in enumerate(sub_result):
    print(f'distance of f1 from centroid of class_{i}: {row}')

one feature with dimension 5 , f1:
 [20 21 22 23 24]
centroid, ctd with 4 class :
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]

sub_result, for each feature we have 4 rows after substraction:
 [[20 20 20 20 20]
 [15 15 15 15 15]
 [10 10 10 10 10]
 [ 5  5  5  5  5]]
the 4 rows of the result are the distance of the feature from the centroid of each class
distance of f1 from centroid of class_0: [20 20 20 20 20]
distance of f1 from centroid of class_1: [15 15 15 15 15]
distance of f1 from centroid of class_2: [10 10 10 10 10]
distance of f1 from centroid of class_3: [5 5 5 5 5]


In [29]:
# to get the eucladian distance, calculate squared substraction
squared_substraction = np.square(sub_result)
print('squared_substraction:\n', squared_substraction)
#### eucladian distance of f1 from each class represented by a vector of 5 dimension
### those 5 values from the vector is summed up tp get a scalar value 
sum_squared_substraction = np.sum(np.square(sub_result), axis=1)
print('sum_squared_substraction:\n', sum_squared_substraction )

squared_substraction:
 [[400 400 400 400 400]
 [225 225 225 225 225]
 [100 100 100 100 100]
 [ 25  25  25  25  25]]
sum_squared_substraction:
 [2000 1125  500  125]


In [30]:
### the abobe example is for a single feature , how to do this for all the features
# can we substract centroids array from the entire feature array? 
# we can not substract different shaped arrays , see the error
print('features.shape', features.shape) # ## features.shape (32, 16)
print('centroids.shape', centroids.shape)  ##centroids.shape (4, 16)
# features - centroids # InvalidArgumentError: Incompatible shapes: [32,16] vs. [4,16] [Op:Sub]
#### substraction is elementwise substraction. so we both the array have to equal shape
z = np.expand_dims(features, axis=1) ## (32, 1, 16)
C =  np.expand_dims(centroids, axis=0) ### (1, 4, 16)
print('np.expand_dims(features, axis=1) :', z.shape)
print('np.expand_dims(centroids, axis=0):', C.shape)
# print('first dimenstion of C:',C[0])
# Now we can substract
sub_z_C = z - C
print('sub_z_C , for each feature 4 results:', sub_z_C.shape)
squred_sum = np.sum(np.square(z- C), axis=2)
print('squred_sum', squred_sum.shape)
print('Eucaldian distance of first feature from the 4 classes:\n',squred_sum[0])

features.shape (32, 16)
centroids.shape (4, 16)
np.expand_dims(features, axis=1) : (32, 1, 16)
np.expand_dims(centroids, axis=0): (1, 4, 16)
sub_z_C , for each feature 4 results: (32, 4, 16)
squred_sum (32, 4)
Eucaldian distance of first feature from the 4 classes:
 [138.6405687   34.1610741   77.21127719 117.537506  ]


In [31]:
# from the data iteration done beforem we will have the last features and the last label_batch
## lets see what is the label for the 0th element in that batch
print('0th element from the last bath retrieved from data iteration previously:\n', features[0])
print('label for the feature_0:\n', label_batch[0])

0th element from the last bath retrieved from data iteration previously:
 tf.Tensor(
[3.7446022 0.5150217 0.        0.        0.6712336 0.        2.7540977
 5.176922  0.        4.8775744 2.0351896 6.470668  7.589819  0.
 0.        0.4365957], shape=(16,), dtype=float32)
label for the feature_0:
 tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)


In [32]:
# Here eucladian distance of the first feature is least from the class_0 

In [33]:
# make the dimensions same for substraction
def euclidean_metric(a, b):
    a = np.expand_dims(a, 1)
    b = np.expand_dims(b, 0)
#     logits = -((a - b)**2).sum(dim=2)
    logits = np.sum(-np.square(a - b), axis=2)
    return logits  

ED_logits = euclidean_metric(features, centroids)
print('ED_logits', ED_logits.shape)
print('ED_logits_sample', ED_logits[0])

ED_logits (32, 4)
ED_logits_sample [-138.6405687   -34.1610741   -77.21127719 -117.537506  ]


In [34]:
### we got the same result only in negative sign 
# these four eucladian values can be taken as softmax 
# to convert it as probability amonghst the four
# Then the max value ( that is why -ve sign will help)
# will represent the class with highest probability

In [35]:
t = tf.range(10)
t = tf.reshape(t, (2, 5))
print(t)
tf.argmax(t, axis=1)

tf.Tensor(
[[0 1 2 3 4]
 [5 6 7 8 9]], shape=(2, 5), dtype=int32)


<tf.Tensor: shape=(2,), dtype=int64, numpy=array([4, 4], dtype=int64)>

In [36]:
# can we get the max value instead of the index
tf.reduce_max(t, 1)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 9])>

In [37]:
# smax = tf.nn.softmax(ED_logits, axis=1)
smax = tf.nn.softmax(ED_logits, )
print('smax.shape:', smax.shape)
class_idx_having_minimum_distance = tf.math.argmax(smax, axis=1)
for i in range(5):    
    print('smax_sample:', smax[i].numpy())
    print('class_idx_having_minimum_distance:', class_idx_having_minimum_distance[i].numpy())

smax.shape: (32, 4)
smax_sample: [4.21824716e-46 1.00000000e+00 2.01156632e-19 6.16702497e-37]
class_idx_having_minimum_distance: 1
smax_sample: [1.00000000e+000 6.53537337e-043 5.75224427e-095 1.32068225e-102]
class_idx_having_minimum_distance: 0
smax_sample: [1.01619355e-71 1.00000000e+00 3.10280417e-79 5.70993531e-97]
class_idx_having_minimum_distance: 1
smax_sample: [2.12776013e-123 2.48075853e-092 1.00000000e+000 2.08047429e-021]
class_idx_having_minimum_distance: 2
smax_sample: [1.87884160e-019 1.00000000e+000 5.70233072e-119 5.26682800e-132]
class_idx_having_minimum_distance: 1


In [38]:
print('label for the feature_0:\n', label_batch[0])
label_indexs = np.argmax(label_batch, axis=1)
print('label_indexs.shape', label_indexs.shape)
print('sample label_indexes',label_indexs[:5])
print('centroids.shape:', centroids.shape)
c = centroids[label_indexs]
print('c.shape:', c.shape)
print(c[0])

label for the feature_0:
 tf.Tensor([0. 1. 0. 0.], shape=(4,), dtype=float32)
label_indexs.shape (32,)
sample label_indexes [1 0 1 2 1]
centroids.shape: (4, 16)
c.shape: (32, 16)
[7.16514893 2.00884033 0.         0.         1.42561722 0.
 2.87550415 8.80388062 0.         6.78546265 0.40732346 6.28099854
 7.50482544 0.         0.         0.03961719]


In [39]:
### To understand this lets take simple example
a = np.arange(12).reshape((4, 3))
print('a:\n',a)
# print(a.shape)
print('a[0]:\n',a[0])
print()
# we create a index whose dimension is higher than a 
b = np.tile([0, 1, 2, 3], (4))
print('b.shape:\n',b.shape)
# b = 16 but a is having 4 rows
print('b:\n',b)
#Notice that each element of b is wihin 0-3 , matching with the max row of a.
# the slicing can be done now as:
print()
print(a[b])

a:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
a[0]:
 [0 1 2]

b.shape:
 (16,)
b:
 [0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3]

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]


In [40]:
# remeber c = centroids[label_indexs]
# only the centroid of the class corresponding to the feature
# earlier we had to expand dimension becuase centroid contained all the 
# four classes
dis = features - c 
print(dis.shape)
euc_dis = tf.norm(features - c, ord='euclidean', axis=1,)
print('euc_dis', euc_dis.shape)
for d, ed in zip(dis[:5], euc_dis[:5]):
    print(f'distance: {d.numpy()}\n eucladian distance:{ed.numpy()}\n')

(32, 16)
euc_dis (32,)
distance: [-3.4205465  -1.4938186   0.          0.         -0.7543836   0.
 -0.12140656 -3.6269588   0.         -1.9078884   1.6278661   0.18966913
  0.08499336  0.          0.          0.39697853]
 eucladian distance:5.844747543334961

distance: [-1.2603054  -0.23462582  0.          0.          0.1492467   0.
  0.605216   -1.0990686   0.         -0.25971603  0.7028725   0.24417925
  0.42120457  0.          0.          0.        ]
 eucladian distance:2.009589672088623

distance: [ 0.7309618  -2.0088403   0.          0.         -1.4256172   0.
 -0.9356859  -0.90286875  0.         -0.47712517 -0.40732345  2.0228868
  1.4808378   0.          0.         -0.03961718]
 eucladian distance:3.8694825172424316

distance: [-0.07835517 -0.02143901  0.          0.          0.29268885  0.
  1.1425757  -0.37526575  0.         -0.67999434  0.69184303  0.35110903
  1.0615816   0.          0.         -0.5192516 ]
 eucladian distance:1.999866008758545

distance: [ 1.2735462   2.254

In [41]:
# # torch.randn, Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution).
# w_init = tf.random.normal([4],0, 1, tf.float32)
# print('w_init Outputs random values from a normal distribution.', w_init)
# w_init = tf.random.normal((4),0, 1, tf.float32)

### shape (4, 1 ) for delta  = 4 rows for the four classes
w_init = tf.random_normal_initializer()
w_init(shape=(4, 1), dtype='float32')

d = tf.Variable(
    initial_value=w_init(shape=(4, 1), dtype='float32'),
    trainable=True,
)
print('d', d)

# w_init = tf.random_normal_initializer()
# self.w = tf.Variable(
#     initial_value=w_init(shape=(input_shape[-1], self.units),
#                          dtype='float32'),
#     trainable=True)

d <tf.Variable 'Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.00879526],
       [-0.01946589],
       [ 0.00327438],
       [-0.00953929]], dtype=float32)>


In [42]:
neg_mask = euc_dis < 1
print('neg_mask', neg_mask)
neg_mask = tf.dtypes.cast(neg_mask, tf.float32)
print('neg_mask', neg_mask)

neg_mask tf.Tensor(
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False], shape=(32,), dtype=bool)
neg_mask tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32,), dtype=float32)


In [43]:

# neg_loss = neg_loss * neg_mask
# InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a bool tensor [Op:Mul]
neg_loss = (d - euc_dis) * neg_mask
print('neg_loss', neg_loss)

neg_loss tf.Tensor(
[[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]], shape=(4, 32), dtype=float32)


In [44]:
pos_mask = euc_dis > 1
print('pos_mask', pos_mask)
pos_mask = tf.dtypes.cast(pos_mask, tf.float32)
print('pos_mask', pos_mask)
pos_loss = (euc_dis - d) * pos_mask
print('pos_mask', pos_loss)

pos_mask tf.Tensor(
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True], shape=(32,), dtype=bool)
pos_mask tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.], shape=(32,), dtype=float32)
pos_mask tf.Tensor(
[[ 5.853543   2.018385   3.8782778  2.0086613  4.374967   2.0525532
   1.7472624  4.395158   2.0699341  1.5119164  2.4587846  4.973954
   2.4292662  2.4587846  4.3594375  1.9662555  4.2746615  2.4587846
   2.4587846  3.8782778  1.6274182  2.4292662  1.6723603  2.4292662
   1.8506229  2.2431033  2.0823798  1.8593949 10.188647   2.4608955
   3.8782778  2.207465 ]
 [ 5.8642135  2.0290556  3.8889484  2.019332   4.3856378  2.0632238
   1.757933   4.4058285  2.0806048  1.5225871  2.4694552  4.984625
   2.4399369  2.4694552  4.370108   1.9769262  4.285332   2.4694552
   2.4694552  3.8889484

In [45]:
tf.reduce_mean(pos_loss, axis=1)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([3.017399 , 3.0280685, 3.0053294, 3.0181422], dtype=float32)>

In [46]:
class BoundaryLoss(tf.keras.layers.Layer):
    def __init__(self, num_labels=train_data.element_spec[1].shape[1], 
                feat_dim = 16):
        super().__init__()
        self.num_labels = num_labels
        self.feat_dim = feat_dim
        # initializing the delta boundary (4,1 shape is for 4 classes 4 number of scaler value)
        w_init = tf.random_normal_initializer()
        self.delta = tf.Variable(
                            initial_value=w_init(shape=(4, 1), dtype='float32'),
                            trainable=True,
                        )
        
    def call(self, features, centroids, labels):
        logits =  euclidean_metric(features, centroids)  
        ######### Why softmax before softplus#########
        smax = tf.nn.softmax(logits, )
        # this is equivallent to predicting the feature belong to which class
        preds = tf.math.argmax(smax, axis=1)
        # This is equivallent to obtaining the max probabiliy of a feature belonging to a calss
        probs = tf.reduce_max(smax, 1)        
        ############################
        # delta =  log(1 + e ^ delta_k) , delta_k =self.delta = parameters for the boundary
        delta = tf.nn.softplus(self.delta)  
#         label_indexs = np.argmax(label_batch, axis=1)
        label_indexs = np.argmax(labels, axis=1)
        # centroids are having only 4 rows , whereas labels are rows equivallent to batch
        # pick-up the centroid for each class 
        # label_index from the data set will have all the classes, 32 for a batch
        # for each class cetroid[class_index] will give the centroid of the calss
        # it is basically : [centroids[class_idx] for class_idx in label_indexes]
        c = centroids[label_indexs]
        # similarly get the delta for each class, 
        # although delta is now randomly intialized 
        # delta parameters will be learned through the training
        d = delta[label_indexs]
        x = features
        # x-c = vector of (32, 16) dimension , euc_dis  = scalar value
        euc_dis = tf.norm(x - c, ord='euclidean', axis=1)        
        ##If axis is None (the default), the input is considered a vector and a 
        ## single vector norm is computed over the entire set of values in the tensor, 
        ## i.e. norm(tensor, ord=ord) is equivalent to norm(reshape(tensor, [-1]), ord=ord). 
        ##If axis is a Python integer, the input is considered a batch of vectors, and axis determines the axis in tensor over which to compute vector norms.
        pos_mask = tf.dtypes.cast(euc_dis > d, tf.int32)
        neg_mask = tf.dtypes.cast(euc_dis < d, tf.int32)
        # euc_dis > d should be ==>1 and euc_dis <= d should be ==>0
        # but the expression here will it retrun True , False or 1 and 0. 
        pos_loss = (euc_dis - d) * pos_mask
        neg_loss = (d - euc_dis) * neg_mask
        loss = tf.reduce_mean(pos_loss, axis=1) + tf.reduce_mean(neg_loss, axis=1)
        
        return loss, delta

## Boundary Loss looks good - every line we have tested and found a logic 

In [47]:
# understand this , a= features(batch_size, 2048) , b = centroids (4, 2048)
def euclidean_metric_torch(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    logits = -((a - b)**2).sum(dim=2)
    return logits

In [48]:
import numpy as np
import tensorflow as tf
a = np.arange(6)
a = a.reshape((2, -1))
print('a:', a)
print('a.shape', a.shape)
b = np.arange(8, 16)
print('b',b)
b = np.reshape(b, (4, -1))
print('b',b)
print('b.shape:',b.shape)
tfa = tf.constant(a)
tfb = tf.constant(b)
print('tfa',tfa)
print('tfb',tfb)
# n = tfa.shape[0]
# m = b.shape[0]


a: [[0 1 2]
 [3 4 5]]
a.shape (2, 3)
b [ 8  9 10 11 12 13 14 15]
b [[ 8  9]
 [10 11]
 [12 13]
 [14 15]]
b.shape: (4, 2)
tfa tf.Tensor(
[[0 1 2]
 [3 4 5]], shape=(2, 3), dtype=int32)
tfb tf.Tensor(
[[ 8  9]
 [10 11]
 [12 13]
 [14 15]], shape=(4, 2), dtype=int32)


In [49]:
print('tf.expand_dims(tfa, 0) :',tf.expand_dims(tfa, 0))
print()
print('tf.expand_dims(tfa, 1) :',tf.expand_dims(tfa, 1))
print()
print('tf.expand_dims(tfa, 1) :',tf.expand_dims(tfa, -1))

tf.expand_dims(tfa, 0) : tf.Tensor(
[[[0 1 2]
  [3 4 5]]], shape=(1, 2, 3), dtype=int32)

tf.expand_dims(tfa, 1) : tf.Tensor(
[[[0 1 2]]

 [[3 4 5]]], shape=(2, 1, 3), dtype=int32)

tf.expand_dims(tfa, 1) : tf.Tensor(
[[[0]
  [1]
  [2]]

 [[3]
  [4]
  [5]]], shape=(2, 3, 1), dtype=int32)


In [50]:
tfa = tf.expand_dims(tfa, 1)
print(f'tf.shape(tfa): {tf.shape(tfa)}')
tfb = tf.expand_dims(tfb, 0)
print(f'tf.shape(tfb): {tf.shape(tfb)}')

tf.shape(tfa): [2 1 3]
tf.shape(tfb): [1 4 2]


In [51]:
n = a.shape[0]
m = b.shape[0]
tfa_broadcast = tf.broadcast_to(tfa, [2, 4, 3])
tf.shape(tfa_broadcast)
print('tfa_broadcast',tfa_broadcast)

tfa_broadcast tf.Tensor(
[[[0 1 2]
  [0 1 2]
  [0 1 2]
  [0 1 2]]

 [[3 4 5]
  [3 4 5]
  [3 4 5]
  [3 4 5]]], shape=(2, 4, 3), dtype=int32)


In [52]:
# 6

# The equivalent function for pytorch expand is tensorflow tf.broadcast_to

# Docs: https://www.tensorflow.org/api_docs/python/tf/broadcast_to

# Share
# Follow
# edited Oct 23, 2021 at 18:22

# M.Innat
# 12.2k66 gold badges3434 silver badges6767 bronze badges
# answered Jan 4, 2019 at 9:12

# funkyyyyyy
# 6111 silver badge22 bronze badges
# Add a comment

# 0

# Tensorflow automatically broadcasts, so in general you don't need to do any of this. Suppose you have a y' of shape 6x2x3 and your x is of shape 2x3, then you can already do y'*x or y'+x will already behave as if you had expanded it. But if for some other reason you really need to do it, then the command in tensorflow is tile:

# y = tf.tile(tf.reshape(x, (1,2,3)), multiples=(6,1,1))
# Docs: https://www.tensorflow.org/api_docs/python/tf/tile

In [53]:
# def euclidean_metric(a, b):
#     n = a.shape[0]
#     m = b.shape[0]
#     a = tf.expand_dims(a, 1)
#     b = tf.expand_dims(b, 0)
#     logits = -((a - b)**2).sum(dim=2)
#     return logits  

In [54]:
centroids = tf.zeros((train_data.element_spec[1].shape[1],   16))
print('centriods initialized:', centroids)
total_labels = tf.zeros(train_data.element_spec[1].shape[1]) # it was 4
# total_labels[2] += 1
# total_labels[2] += 1
print('total_labels initialized:', total_labels)


centriods initialized: tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(4, 16), dtype=float32)
total_labels initialized: tf.Tensor([0. 0. 0. 0.], shape=(4,), dtype=float32)


In [55]:
a = tf.expand_dims(features, 1)
b = tf.expand_dims(centroids, 0)
dis = a - b
squred_dis = tf.math.square(a - b)
logits = tf.math.reduce_sum(-squred_dis, axis=2)
logits

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[-176.72165, -176.72165, -176.72165, -176.72165],
       [-309.08383, -309.08383, -309.08383, -309.08383],
       [-318.02924, -318.02924, -318.02924, -318.02924],
       [-251.50241, -251.50241, -251.50241, -251.50241],
       [-374.17014, -374.17014, -374.17014, -374.17014],
       [-310.32132, -310.32132, -310.32132, -310.32132],
       [-259.6144 , -259.6144 , -259.6144 , -259.6144 ],
       [-376.63464, -376.63464, -376.63464, -376.63464],
       [-256.6905 , -256.6905 , -256.6905 , -256.6905 ],
       [-223.96918, -223.96918, -223.96918, -223.96918],
       [-316.77362, -316.77362, -316.77362, -316.77362],
       [-208.27956, -208.27956, -208.27956, -208.27956],
       [-165.30278, -165.30278, -165.30278, -165.30278],
       [-316.77362, -316.77362, -316.77362, -316.77362],
       [-375.8573 , -375.8573 , -375.8573 , -375.8573 ],
       [-309.35577, -309.35577, -309.35577, -309.35577],
       [-371.0056 , -371.0056 , -371.00

In [56]:
def euclidean_metric(a, b):
    a = tf.expand_dims(a, 1)
    b = tf.expand_dims(b, 0)
#     logits = -((a - b)**2).sum(dim=2)
    logits = tf.math.reduce_sum(-tf.math.square(a - b), axis=2)
    return logits
euclidean_metric(features, centroids)

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[-176.72165, -176.72165, -176.72165, -176.72165],
       [-309.08383, -309.08383, -309.08383, -309.08383],
       [-318.02924, -318.02924, -318.02924, -318.02924],
       [-251.50241, -251.50241, -251.50241, -251.50241],
       [-374.17014, -374.17014, -374.17014, -374.17014],
       [-310.32132, -310.32132, -310.32132, -310.32132],
       [-259.6144 , -259.6144 , -259.6144 , -259.6144 ],
       [-376.63464, -376.63464, -376.63464, -376.63464],
       [-256.6905 , -256.6905 , -256.6905 , -256.6905 ],
       [-223.96918, -223.96918, -223.96918, -223.96918],
       [-316.77362, -316.77362, -316.77362, -316.77362],
       [-208.27956, -208.27956, -208.27956, -208.27956],
       [-165.30278, -165.30278, -165.30278, -165.30278],
       [-316.77362, -316.77362, -316.77362, -316.77362],
       [-375.8573 , -375.8573 , -375.8573 , -375.8573 ],
       [-309.35577, -309.35577, -309.35577, -309.35577],
       [-371.0056 , -371.0056 , -371.00

In [57]:
# how to assign a new value to index position of a tensor
total_labels # is a tensor 

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [58]:
tf.unstack(total_labels) # unstack it to a list

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]

In [59]:
lst = tf.unstack(total_labels)
## now assign values to index position of the list
lst =  [1, 0, 0, 0]

In [60]:
#convert them to tensor again
newtf = tf.stack(lst)
newtf

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 0, 0, 0])>

In [78]:
for batch in train_data: # Remember <BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
    logseq_batch, label_batch = batch
    # (32, 32, 64), (32, 4)
    features = log_classifier(logseq_batch, extract_feature=True )
    # (32, 16) features - 32 sequence of line each haaving 64 characrers
    # produces a feaure vector of dimension 16. 
    for i in range(len(label_batch)): # (32, 4) --> here length is 32
        label = label_batch[i] # label looks like [0 0 0 1]
        numeric_label = np.argmax(label) # index position of the label = 3 , so it is actually class =3
        ##total_labels = [0 0 0 0] each col representing a class 
        ## count the number for each class
        total_labels_lst = tf.unstack(total_labels)
        total_labels_lst[numeric_label] += 1 
        total_labels = tf.stack(total_labels_lst)
        centroids_lst = tf.unstack(centroids)
        centroids_lst[numeric_label] += features[i]
        centroids = tf.stack(centroids_lst)
        # each row index in the centroid array is a class
        # we add first identify the feature belonging to which class by the numeric_label
        # Then add all the features belonging to the class in the corresponding row of the centroid array

# ### shape of centroids is (4, 16) whereas shape of total_labels is (1, 4)
# ### reshape the total_labels as 4,1 ==> [[0], [0], [0], [0]]==> 4 rows 
# ## so that we can divide the centroids array by the total_labels
total_label_reshaped = np.reshape(total_labels, (train_data.element_spec[1].shape[1], 1))
centroids /= total_label_reshaped
print('centroids:',centroids)
print('total_labels:',total_label_reshaped)

centroids: tf.Tensor(
[[2.2333534  4.336265   0.         0.         2.2633772  0.
  2.8577154  4.883939   0.         3.6227775  1.0883478  1.5947356
  2.1637454  0.         0.         0.        ]
 [3.5870562  1.0056759  0.         0.         0.7136998  0.
  1.439549   4.4074416  0.         3.396974   0.2039164  3.1444259
  3.7571042  0.         0.         0.01983335]
 [0.09745792 0.01073291 0.         0.         0.7897265  0.
  2.3192055  0.38710183 0.         1.9512906  3.4741373  3.5969954
  3.7934008  0.         0.         2.1638904 ]
 [0.2828121  0.1923533  0.         0.         0.17389129 0.
  3.8500261  1.5355862  0.         0.35170245 4.220116   2.0535035
  5.7093763  0.         0.         0.18759584]], shape=(4, 16), dtype=float32)
total_labels: [[1600.]
 [1600.]
 [1600.]
 [1600.]]


In [66]:
label_indexes = tf.math.argmax(label_batch, axis=1)
label_indexes

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 0, 0, 1, 3, 0, 0, 0, 1, 3, 3, 3, 1, 2, 1, 1, 3, 1, 0, 1, 0, 2,
       0, 2, 2, 2, 1, 0, 1, 3, 2, 2], dtype=int64)>

In [68]:
label_indexes.numpy()

array([1, 0, 0, 1, 3, 0, 0, 0, 1, 3, 3, 3, 1, 2, 1, 1, 3, 1, 0, 1, 0, 2,
       0, 2, 2, 2, 1, 0, 1, 3, 2, 2], dtype=int64)

In [85]:
c = centroids
c

<tf.Tensor: shape=(4, 16), dtype=float32, numpy=
array([[2.2333534 , 4.336265  , 0.        , 0.        , 2.2633772 ,
        0.        , 2.8577154 , 4.883939  , 0.        , 3.6227775 ,
        1.0883478 , 1.5947356 , 2.1637454 , 0.        , 0.        ,
        0.        ],
       [3.5870562 , 1.0056759 , 0.        , 0.        , 0.7136998 ,
        0.        , 1.439549  , 4.4074416 , 0.        , 3.396974  ,
        0.2039164 , 3.1444259 , 3.7571042 , 0.        , 0.        ,
        0.01983335],
       [0.09745792, 0.01073291, 0.        , 0.        , 0.7897265 ,
        0.        , 2.3192055 , 0.38710183, 0.        , 1.9512906 ,
        3.4741373 , 3.5969954 , 3.7934008 , 0.        , 0.        ,
        2.1638904 ],
       [0.2828121 , 0.1923533 , 0.        , 0.        , 0.17389129,
        0.        , 3.8500261 , 1.5355862 , 0.        , 0.35170245,
        4.220116  , 2.0535035 , 5.7093763 , 0.        , 0.        ,
        0.18759584]], dtype=float32)>

In [86]:
tf.gather(centroids, indices=label_indexes)

<tf.Tensor: shape=(32, 16), dtype=float32, numpy=
array([[3.5870562 , 1.0056759 , 0.        , 0.        , 0.7136998 ,
        0.        , 1.439549  , 4.4074416 , 0.        , 3.396974  ,
        0.2039164 , 3.1444259 , 3.7571042 , 0.        , 0.        ,
        0.01983335],
       [2.2333534 , 4.336265  , 0.        , 0.        , 2.2633772 ,
        0.        , 2.8577154 , 4.883939  , 0.        , 3.6227775 ,
        1.0883478 , 1.5947356 , 2.1637454 , 0.        , 0.        ,
        0.        ],
       [2.2333534 , 4.336265  , 0.        , 0.        , 2.2633772 ,
        0.        , 2.8577154 , 4.883939  , 0.        , 3.6227775 ,
        1.0883478 , 1.5947356 , 2.1637454 , 0.        , 0.        ,
        0.        ],
       [3.5870562 , 1.0056759 , 0.        , 0.        , 0.7136998 ,
        0.        , 1.439549  , 4.4074416 , 0.        , 3.396974  ,
        0.2039164 , 3.1444259 , 3.7571042 , 0.        , 0.        ,
        0.01983335],
       [0.2828121 , 0.1923533 , 0.        , 0.    

In [62]:
# In context of deep learning the logits layer means the layer that feeds in to softmax (or other such normalization). The output of the softmax are the probabilities for the classification task and its input is logits layer. The logits layer typically produces values from -infinity to +infinity and the softmax layer transforms it to values from 0 to 1.

# Historical Context

# Where does this term comes from? In 1930s and 40s, several people were trying to adapt linear regression to the problem of predicting probabilities. However linear regression produces output from -infinity to +infinity while for probabilities our desired output is 0 to 1. One way to do this is by somehow mapping the probabilities 0 to 1 to -infinity to +infinity and then use linear regression as usual. One such mapping is cumulative normal distribution that was used by Chester Ittner Bliss in 1934 and he called this "probit" model, short for "probability unit". However this function is computationally expensive while lacking some of the desirable properties for multi-class classification. In 1944 Joseph Berkson used the function log(p/(1-p)) to do this mapping and called it logit, short for "logistic unit". The term logistic regression derived from this as well.

# The Confusion

# Unfortunately the term logits is abused in deep learning. From pure mathematical perspective logit is a function that performs above mapping. In deep learning people started calling the layer "logits layer" that feeds in to logit function. Then people started calling the output values of this layer "logit" creating the confusion with logit the function.